In [36]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.load_bold_data import load_multisite_data
import numpy as np, torch
from torch.utils.data import Dataset
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')


In [37]:
tf.random.set_seed(42)

In [38]:
logging = True

In [39]:
n_rois = 18
n_splits = 5
n_repeats = 1

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, GlobalAveragePooling3D, Conv2D, Input, GlobalAveragePooling2D, MaxPooling2D, Flatten, Dense, InputLayer, Dropout, TimeDistributed, LSTM


timesteps = 172    # número de imágenes en la secuencia
height = 18
width = 18
channels = 1

def create_model():
  inputs = Input(shape=(timesteps, height, width, channels))

  x = Conv3D(32, (3,3,3), activation='relu')(inputs)
  x = MaxPooling3D((1,2,2))(x)

  x = Conv3D(64, (3,3,3), activation='relu')(x)
  x = GlobalAveragePooling3D()(x)

  x = Dense(96, activation='relu')(x)

  outputs = Dense(1, activation='sigmoid')(x)

  model = Model(inputs, outputs)
   
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy', 'precision', 'recall'])
  # Resumen del modelo
  return model



In [41]:
X = joblib.load(f"../data/fcm-fisher/X_172_{n_rois}.joblib")
y = joblib.load(f"../data/fcm-fisher/y_172_{n_rois}.joblib")

# Display the shape of the concatenated array
print(X.shape)

(177, 172, 18, 18)


In [42]:
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the number of folds for cross-validation


skf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats)
# Initialize lists to store results
accuracy_scores = []
precision_scores = []
recall_scores = []

# Define Early Stopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=1e-4,
    mode='max',
    verbose=1,
    restore_best_weights=True
)

X = np.array(X)
y = np.array(y)
# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print(f"Fold {fold+1}/{n_splits*n_repeats}")

    # Split data into training and validation sets for the current fold
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create a new model for each fold to ensure independent training
    model = create_model()

    # Define a ModelCheckpoint callback to save the best model weights
    checkpoint_filepath = f'tmp_keras_checkpoint/best_model_fold_{fold+1}.keras'
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=0
    )

    # Train the model with the callbacks
    history = model.fit(
        X_train,
        y_train,
        epochs=100, # Increased epochs to allow Early Stopping to trigger
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[model_checkpoint_callback, early_stopping_callback],
        verbose=0
    )

    # Evaluate the model on the validation set
    # Load the best model weights before evaluating
    best_model = create_model() # Create a new model instance with the same architecture
    best_model.load_weights(checkpoint_filepath)
    loss, accuracy, precision, recall = best_model.evaluate(X_val, y_val, verbose=0)
    if logging:
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation Precision: {precision:.4f}")
        print(f"Validation Recall: {recall:.4f}")
    # Store the scores
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)

# Print average scores across all folds
print("\nAverage Cross-Validation Scores:")
print(f"Accuracy: {np.mean(accuracy_scores):.4f} +- {np.std(accuracy_scores):.4f}")
print(f"Precision: {np.mean(precision_scores):.4f} +- {np.std(precision_scores):.4f}")
print(f"Recall: {np.mean(recall_scores):.4f} +- {np.std(recall_scores):.4f}")
df_results = pd.DataFrame(
    {"accuracy": accuracy_scores,
    "precision": precision_scores,
    "recall": recall_scores}
)
df_results.to_csv(f"tmp_results/results_{n_rois}_rois.csv", index=None)

Fold 1/5
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation Accuracy: 0.5000
Validation Precision: 0.0000
Validation Recall: 0.0000
Fold 2/5
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation Accuracy: 0.5000
Validation Precision: 0.0000
Validation Recall: 0.0000
Fold 3/5
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation Accuracy: 0.5143
Validation Precision: 0.5143
Validation Recall: 1.0000
Fold 4/5
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation Accuracy: 0.5143
Validation Precision: 0.5143
Validation Recall: 1.0000
Fold 5/5
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 1.
Validation Accuracy: 0.5143
Validation Precision: 0.5143
Validation Recall: 1.0000

Average Cross-Validation Scores:
Accuracy: 0.5086 +- 0.0070
Precision: 0.3086 +- 0.2519
Recall: 0.6000 +- 0.4899
